# Creating a new regionalized water stress LCIA method

Thanks to Paola for this idea! CF map from [WRI](https://www.wri.org/data/aqueduct-water-stress-projections-data).

In [ ]:
import fiona
import bw2data as bd
import bw2regional as bwr

In [ ]:
bd.projects.set_current("LC IMPACT case study")

In [ ]:
WATER_RESOURCE = [
    ("Water, lake", ("natural resource", "in water")),
    ("Water, river", ("natural resource", "in water")),
    (
        "Water, turbine use, unspecified natural origin",
        ("natural resource", "in water"),
    ),
    ("Water, unspecified natural origin", ("natural resource", "in water")),
    ("Water, cooling, unspecified natural origin", ("natural resource", "in water")),
    ("Water, unspecified natural origin", ("natural resource", "in ground")),
    ("Water, well, in ground", ("natural resource", "in water")),
]

WATER_RELEASE = [
    ("Fresh water (obsolete)", ("water", "surface water")),
    ("Water", ("water",)),
    ("Water", ("water", "surface water")),
    ("Water", ("water", "ground-")),
]

In [ ]:
fp = "/srv/data/vectors/aqueduct_ws.gpkg"

In [ ]:
def import_aqueduct_lcia(biosphere="biosphere3", column='ws4038cr', id_field="BasinID"):
    bio = bd.Database(biosphere)

    gc = bwr.geocollections["watersheds-aqueduct"] = {
        "filepath": fp,
        "url": "https://www.wri.org/data/aqueduct-water-stress-projections-data",
        "field": "BasinID",        
    }

    method = bd.Method(("Aqueduct Water Stress", "WS4038"))
    method.register(
        geocollections=["watersheds-aqueduct"],
        unit = "Water stress",
        description="""Water stress under a future scenario. 
        Map downloaded from World Resources Institute, we use values for scenario ws4038cr.""",
    )

    positive = [bio.get(name=x, categories=y) for x, y in WATER_RESOURCE]
    negative = [bio.get(name=x, categories=y) for x, y in WATER_RELEASE]

    cfs = []

    with fiona.Env():
        with fiona.open(fp) as src:
            for feat in src:
                for node in positive:
                    cfs.append((
                        node.key,
                        feat["properties"][column],
                        ("watersheds-aqueduct", feat["properties"][id_field]),
                    ))
                for node in negative:
                    cfs.append((
                        node.key,
                        -1 * feat["properties"][column],
                        ("watersheds-aqueduct", feat["properties"][id_field]),
                    ))

    method.write(cfs)
    return method

In [ ]:
import_aqueduct_lcia()

Test it out

In [ ]:
for inv_gc in ['world', 'BCN', 'ALM']:
    bwr.remote.calculate_intersection(inv_gc, 'watersheds-aqueduct')

In [ ]:
for inv_gc in ['world', 'BCN', 'ALM']:
    bwr.remote.intersection_as_new_geocollection(
        inv_gc, 
        'watersheds-aqueduct', 
        f'{inv_gc} - watersheds-aqueduct',
    )

In [ ]:
for inv_gc in ['world', 'BCN', 'ALM']:
    bwr.remote.calculate_rasterstats(
        f'{inv_gc} - watersheds-aqueduct',
        'weighted-pop-density',
    )

In [ ]:
for inv_gc in ['world', 'BCN', 'ALM']:
    bwr.remote.rasterstats_as_xt(    
        f'{inv_gc} - watersheds-aqueduct',
        'weighted-pop-density',
        f'{inv_gc} - watersheds-aqueduct - xt',
    )

In [ ]:
lca = bwr.ExtensionTablesLCA(
    demand={('meal', 'meal'): 1},
    method=("Aqueduct Water Stress", "WS4038"),
    xtable='world - watersheds-aqueduct - xt'
)
lca.lci()
lca.lcia()

In [ ]:
lca.geodataframe_ia_spatial_scale(cutoff=1e-3).explore(column='score_rel')